In [ ]:
# import os
import json
import itertools
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import json
import pandas as pd
import threading


class getDataFromDBPedia():
    def __init__(self,typeString,workFolder):
        self.typeString=typeString
        self.workFolder=workFolder
        self.status={}
        self.invidualItemStatus='incomplete'
        self.invidualItemLastLetter='AA'
        self.invidualItemLastFile=0
        
        # We will write a dummy JSON status file if not already present, so that it need not be ahndled every time
        if os.path.exists(workFolder + '/status.json'):
            print("We already have a prior status")
        else:
            self.status['type']=self.typeString
            self.status['individualItemStatus']={}
            self.status['individualItemStatus']['status']=self.invidualItemStatus
            self.status['individualItemStatus']['lastLetter']=self.invidualItemLastLetter
            self.status['individualItemStatus']['lastFile']=self.invidualItemLastFile
            with open(workFolder + '/status.json','w') as f:
                f.write(json.dumps(self.status))
        
    def getStatusInividualItems(self):
        # Check whether we already have downloaded some data in the existing workfolder
        with open(self.workFolder + '/status.json') as f:
            self.status = json.load(f)
        if( 'type' in self.status):
            if(self.status['type'] != self.typeString):
                return False
        if( 'individualItemStatus' in self.status):
            # This means that we have prior downloaded
            self.invidualItemStatus=self.status['individualItemStatus']['status']
            self.invidualItemLastLetter=self.status['individualItemStatus']['lastLetter']
            self.invidualItemLastFile=self.status['individualItemStatus']['lastFile']
            
        return True
    
    def getIndividualItems(self):
        firstCharacterString='ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@_#$^&='
        characterList=list(itertools.product(firstCharacterString,firstCharacterString))
        characterList=[str(x[0]) + str(x[1]) for x in characterList]
        fullCompanyData=pd.DataFrame()
        
        for firstThreeCharacters in characterList[characterList.index(self.invidualItemLastLetter):]:
            query="""SELECT distinct ?orgObject ?orgLabel
            WHERE {
            ?orgObject rdf:type placeholderObjectType .
            ?orgObject rdfs:label ?orgLabel .
            filter langMatches(lang(?orgLabel),"en")
            filter(regex(REPLACE(?orgLabel, '"', '', "i"),"^%s","i"))
            }
            """ % format(firstThreeCharacters)
            query=query.replace('placeholderObjectType',self.typeString)
            data=self.returnDF(self.get_dbpedia_sparql_data(query))
            print("{} : {}".format(firstThreeCharacters,data.shape[0]))
            fullCompanyData=pd.concat([fullCompanyData,data])
            if(fullCompanyData.shape[0] > 50000):
                print("Writing data to file {}".format(self.invidualItemLastFile))
                fullCompanyData.to_csv(self.workFolder + '/FullData{}.csv'.format(self.invidualItemLastFile),index=False,encoding='utf-8')
                self.invidualItemLastFile=self.invidualItemLastFile + 1
                fullCompanyData=pd.DataFrame()
                
                # We will refresh the status JSON and write it back
                self.invidualItemStatus='incomplete'
                self.invidualItemLastLetter=firstThreeCharacters
                self.status['individualItemStatus']['status']=self.invidualItemStatus
                self.status['individualItemStatus']['lastLetter']=self.invidualItemLastLetter
                self.status['individualItemStatus']['lastFile']=self.invidualItemLastFile
                with open(self.workFolder + '/status.json','w') as f:
                    f.write(json.dumps(self.status))
        
        print("Writing data to file {}".format(self.fileCounter))
        fullCompanyData.to_csv(self.workFolder + '/FullData{}.csv'.format(self.invidualItemLastFile),index=False,encoding='utf-8')
        self.invidualItemStatus='complete'
        self.invidualItemLastLetter=firstThreeCharacters
        self.invidualItemLastFile=fileCounter
        self.status['individualItemStatus']['status']=self.invidualItemStatus
        self.status['individualItemStatus']['lastLetter']=self.invidualItemLastLetter
        self.status['individualItemStatus']['lastFile']=self.invidualItemLastFile
        with open(self.workFolder + '/status.json','w') as f:
            f.write(json.dumps(self.status))

    def get_dbpedia_sparql_data(self,query):
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setReturnFormat(JSON)
        sparql.setQuery(query)  # the previous query as a literal string
        return sparql.query().convert()

    def returnDF(self,data):
        dataArray=[]
        colNames=data['head']['vars']
        for row in data['results']['bindings']:
            rowArray=[]
            for colName in colNames:
                rowArray.append(row[colName]['value'])
            dataArray.append(rowArray)

        dataArray=pd.DataFrame(dataArray,columns=colNames)
        return(dataArray)
            
if __name__=="__main__":
    ThingObject=getDataFromDBPedia('owl:Thing','/home/anantgupta/Documents/Programming/dbpedia/Thing')
    if(ThingObject.getStatusInividualItems()==True):
        ThingObject.getIndividualItems()
    
# After we have got the ORG LIST, we will now be exporting the entire ORG Data Structure available in DBPEDIA
import os
fullData=pd.DataFrame()
for x in os.listdir('/home/anantgupta/Documents/Programming/dbpedia/Thing'):
    fullData=pd.concat([fullData,pd.read_csv('/home/anantgupta/Documents/Programming/dbpedia/Thing/'+str(x))])
    print(fullData.shape)

fullData.to_csv('/home/anantgupta/Documents/Programming/dbpedia/Thing/ThingFullList.txt',index=False)

# Get the details around each Thing
orgFullData=pd.read_csv('/home/anantgupta/Documents/Programming/dbpedia/Thing/ThingFullList.txt')

orgDetailedData=pd.DataFrame(columns=['org','property','value'])
batch='A'
batchCounter=0
for iterCounter,x in enumerate(orgFullData['orgObject'].values):
    query="""
    SELECT ?property ?hasValue
    WHERE {
      <%s> ?property ?hasValue
    }
    """ % (x)
    tempData=returnDF(get_dbpedia_sparql_data(query))
    orgDetailedData=pd.concat([orgDetailedData,pd.DataFrame(zip([x] * tempData.shape[0],tempData['property'].values,tempData['hasValue'].values),columns=['org','property','value'])])    
    if(iterCounter % 1000 ==0):
        orgDetailedData.to_csv('/home/anantgupta/Documents/Programming/dbpedia/Thing/' + batch+'_'+str(batchCounter)+'.csv',index=False,encoding='utf-8')
        print("Finished counter {} and batch {}".format(batchCounter,batch))
        orgDetailedData=pd.DataFrame(columns=['org','property','value'])
        batchCounter=batchCounter+1

In [4]:
# import os
import json
import itertools
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import json
import pandas as pd
import threading
import os

# Get the list of companies for which we need data
def get_dbpedia_sparql_data(query):
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setReturnFormat(JSON)
        sparql.setQuery(query)  # the previous query as a literal string
        return sparql.query().convert()

def returnDF(data):
    dataArray=[]
    colNames=data['head']['vars']
    for row in data['results']['bindings']:
        rowArray=[]
        for colName in colNames:
            rowArray.append(row[colName]['value'])
        dataArray.append(rowArray)
    dataArray=pd.DataFrame(dataArray,columns=colNames)
    return(dataArray)

In [40]:
# Get the list of companies which have location
companyListQuery='select distinct ?a where {?a rdf:type <http://dbpedia.org/ontology/Company> . ?a <http://dbpedia.org/ontology/country> ?y }'
companyList=returnDF(get_dbpedia_sparql_data(companyListQuery))
companyList.columns=['CompanyName']

# We will now get all the data for each of the company with Location
fileCounter=0
FullCompanyData=pd.DataFrame(columns=['property','hasValue','CompanyName'])
for curCompanyName in companyList['CompanyName'].values:
    companyQuery='''SELECT ?property ?hasValue
    WHERE {
      { <%s> ?property ?hasValue }
      UNION
      { ?hasValue ?property <%s> }
    }''' % (curCompanyName,curCompanyName)
    curCompanyData=returnDF(get_dbpedia_sparql_data(companyQuery))
    curCompanyData['Company']=curCompanyName
    curCompanyData.columns=['property','hasValue','CompanyName']
    FullCompanyData=pd.concat([FullCompanyData,curCompanyData])
    if(FullCompanyData.shape[0] > 100000):    
        FullCompanyData.to_csv('/home/ubuntu/anant/data/CompanyDataWithLocation_{}.txt'.format(fileCounter),index=False,encoding='utf-8')
        fileCounter=fileCounter + 1
        FullCompanyData=pd.DataFrame(columns=['property','hasValue','CompanyName'])
        print("Writtend to file {}".format(fileCounter-1))

FullCompanyData.to_csv('/home/ubuntu/anant/data/CompanyDataWithLocation_{}.txt'.format(fileCounter),index=False,encoding='utf-8')
fileCounter=fileCounter + 1
FullCompanyData=pd.DataFrame(columns=['property','hasValue','CompanyName'])
print("Written to file {}".format(fileCounter-1))

Writtend to file 0
Writtend to file 1
Writtend to file 2
Writtend to file 3
Writtend to file 4
Writtend to file 5
Writtend to file 6
Writtend to file 7
Written to file 8


In [94]:
# Get the predicate data for all the RHS
predicateList=pd.read_csv('/home/ubuntu/anant/data/predicateListJoins.txt',header=-1)
predicateList.columns=['property']
predicateList['property']=predicateList['property'].map(lambda x : str(x).replace('<','').replace('>',''))

# Complete list of all predicates
predicateValues=[]

for curFileName in os.listdir('/home/ubuntu/anant/data/'):
    if('CompanyDataWithLocation_' in curFileName ):
        FullCompanyData=pd.read_csv('/home/ubuntu/anant/data/{}'.format(curFileName))
        tempValues=list(FullCompanyData.merge(predicateList,left_on='property',right_on='property',how='inner')['hasValue'].unique())
        predicateValues=predicateValues + tempValues
        
predicateValues=list(set(predicateValues))

In [100]:
predicateData=pd.DataFrame(columns=['property','hasValue','Predicate'])
fileCounter=0

for curPredicate in predicateValues:
    predicateQuery='''SELECT ?property ?hasValue
    WHERE {
      { <%s> ?property ?hasValue }
      UNION
      { ?hasValue ?property <%s> }
    }''' % (curPredicate,curPredicate)
    try:
        curpredicateData=returnDF(get_dbpedia_sparql_data(predicateQuery))
        curpredicateData['Predicate']=curPredicate
        curpredicateData.columns=['property','hasValue','Predicate']
        predicateData=pd.concat([predicateData,curpredicateData])
    except:
        print("Error for {}".format(curPredicate))
    if(predicateData.shape[0] > 100000):    
        predicateData.to_csv('/home/ubuntu/anant/data/PredicateDataWithLocation_{}.txt'.format(fileCounter),index=False,encoding='utf-8')
        fileCounter=fileCounter + 1
        print("Written to file {} with shape {}".format(fileCounter-1,predicateData.shape[0]))
        predicateData=pd.DataFrame(columns=['property','hasValue','Predicate'])

predicateData.to_csv('/home/ubuntu/anant/data/PredicateDataWithLocation_{}.txt'.format(fileCounter),index=False,encoding='utf-8')
fileCounter=fileCounter + 1
print("Written to file {} with shape {}".format(fileCounter-1,predicateData.shape[0]))
predicateData=pd.DataFrame(columns=['property','hasValue','Predicate'])

Error for Stock photos
Error for Construction & Industrial Equipment Rentals and Sales
Error for Centro Watt Trust
Error for GPS tracking devices
Error for Pet station refill bag rolls, Individual resident scoop bag dispensers,
Written to file 0 with shape 106150
Error for Stock video, audio and graphics
Error for EDMBiz conference, EDM festivals, music publishing
Written to file 1 with shape 103914
Error for Documents  Membership, Home Equity, Business
Error for oil and gas
Error for Life insurance
Written to file 2 with shape 106419
Error for Centro Direct Property Fund International
Written to file 3 with shape 109791
Error for No Yolk noodles, Wacky Mac Pasta, Wacky Mac & Cheese
Error for Recommendation Engine
Written to file 4 with shape 107846
Written to file 5 with shape 101556
Error for Web services
Error for ShoutMD, Teoxane, PRN Physician Recommended Nutriceutials, Skin Essentials, Alphaeon Beauty, AlphaeonMD, iTrace
Written to file 6 with shape 100194
Error for Centro Direct